In [32]:
import json
import matplotlib.pyplot as plt
import numpy as np
import warnings
import datetime
import argparse
import threading
import multiprocessing
import os


from tqdm import tqdm

from GBP_Simulations.GBP.data import DataGenerator
from GBP_Simulations.GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_SYNC_ACCELERATED_UPDATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL
from GBP_Simulations.GBP.utilities import HiddenPrints
from GBP_Simulations.GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

### MITb Dataset

In [33]:
# TRIES = 10

# for it in range(TRIES):

#     file_path = 'GBP_Simulations/GBP/Raw_Datasets/data/input_MITb_g2o.g2o'
#     data_gen.generate_SLAM_dataset_PSD(file_path=file_path)

#     dataset = 'input_MITb_g2o'
#     filepath_n = 'GBP_Simulations/GBP/Raw_Datasets/gbp_data'
#     factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
#     marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

#     A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)

#     sync_convergence_threshold = 1*10**-0 # convergence threshold
#     convergence_type = 'all' #all
#     simulation_convergence_threshold = 1*10**-10

#     P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
#                                                                                                         b, 
#                                                                                                         max_iter=1_000, 
#                                                                                                         mae=True if convergence_type == 'mae' else False, 
#                                                                                                         convergence_threshold=simulation_convergence_threshold,
#                                                                                                         show=True)
#     final_mean_sync = list(mu_i)

#     if iteration < 1_000:
#         break
#     else:
#         iteration = float("inf")
#         print("=========== RESTART ===========")

In [34]:
file_path = 'GBP_Simulations/GBP/Raw_Datasets/data/input_MITb_g2o.g2o'
data_gen.generate_SLAM_dataset_PSD(file_path=file_path)

dataset = 'input_MITb_g2o'
filepath_n = 'GBP_Simulations/GBP/Raw_Datasets/gbp_data'
factor_data = os.path.join(filepath_n, f'{dataset}_factor_data.txt')
marginal_data = os.path.join(filepath_n, f'{dataset}_marginal_data.txt')

A, b = data_gen.fetch_SLAM_dataset(file_path_factor=factor_data, file_path_marginal=marginal_data)

In [43]:
b_prime = 336309.843505 * b # 0.40878665*1_000_000 * b

In [44]:
sync_convergence_threshold = 1*10**-0 # convergence threshold
convergence_type = 'all' #all
simulation_convergence_threshold = 1*10**-10

P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, 
                                                                                                    b_prime, 
                                                                                                    max_iter=1_000, 
                                                                                                    mae=True if convergence_type == 'mae' else False, 
                                                                                                    convergence_threshold=simulation_convergence_threshold,
                                                                                                    show=True)
final_mean_sync = list(mu_i)

iteration: 1
18.281102984714565
-----
iteration: 2
0.2811075536435336
-----
iteration: 3
0.0328539350439652
-----
iteration: 4
0.003790572901161725
-----
iteration: 5
0.0004113133036612744
-----
iteration: 6
2.825939121027719e-05
-----
iteration: 7
3.3885646828536737e-06
-----
iteration: 8
0.0
-----


### Hardware

In [45]:
ASYNC_ITER = 10
NODE_UPDT_PE = 137
PEs = 3

async_convergence_threshold = 1*10**-5

In [46]:
sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b_prime, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean_sync, max_iter=100, starting_mae=10, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 0
10.00000000000804
-----
iteration: 1
10.00000000000804
-----
iteration: 2
5.391682958006924
-----
iteration: 3
2.869989808405867
-----
iteration: 4
1.4254237262278038
-----
iteration: 5
0.7972421709481154
-----
iteration: 6
0.4250482090665283
-----
iteration: 7
0.2350404849652576
-----
iteration: 8
0.038899602074098934
-----
iteration: 9
0.01476576561843517
-----
iteration: 10
0.010577140707982144
-----
iteration: 11
0.0035722690698497496
-----
iteration: 12
0.0004908474753088349
-----
iteration: 13
0.0004173914079591927
-----
iteration: 14
0.00040657460871368594
-----
iteration: 15
0.0004026142201982334
-----
iteration: 16
1.0533008361552513e-08
-----
-------------- ITERATION = 1 => Streams = 17 --------------
iteration: 0
10.00000000000804
-----
iteration: 1
10.00000000000804
-----
iteration: 2
5.03420531547568
-----
iteration: 3
2.4626457552306937
-----
iteration: 4
1.0252342571371886
-----
iteration: 5
0.41063891931601426
-----
iteration: 6
0.16184846023589897
-----
it